# TextRank 알고리즘 적용 위한 스텝
1. 텍스트 수집 (완)
2. 전처리
  - 불용어 처리 (re 사용)
3. tf-idf 통해 문서 내 특정 단어의 중요도 분석
  - 텍스트를 문장 단위로 분리
  - koNLPY twitter 활용해 형태소 태깅 + 조사 및 어간 분리 통해 명사 반환
  - TfidfVectorizer()와 CountVectorizer() 활용하여 tf, df 계산
    - 이때 word2vec, doc2vec 등을 결합하여 문서 간 유사도 계산 가능한 점 참고하기

4. tf-idf 과정을 통해 얻은 문장(단어) 중요도를 활용하여 ranking, 요약할 문장(단어) 수만큼 출력

공지 파일 결측치 41개 어떻게 처리할 것인가? --> 이미지의 손상으로 인해 데이터를 보완할 수 없기 때문에 삭제하기로 결정
- 만약 HUFS 공지 검색을 통해 보완할 수 있다면 최대한 보완할 것인가? 논의하기
- 보완 +로그 남기기 + 더이상 보완할 수 없을 경우 '직접 접속해서 확인해주세요.' 출력하기

2024.04.19
- 공지 파일 결측치 중 보완 가능한 23개 내용 보충 완료 후 재업로드
- 학사 인덱스 보충완료

### 할 순서(0425 회의 이후~)
1. 전번 등 임의 전처리 해서 데이터 전처리
- copy 시 코로나 관련 공지 빼기 (문법이 파괴되지 않는 한에서 한글 표현만 최대한 남기기)
2. kospacing 적용 _ 띄어쓰기 적용
3. 형태소 분석기 (Okt())로 형태소 단위 분리
3-1. 한글자 제거?
4. 불용어 리스트 가져와서 전처리(조사 등많이 제거될 것)
5. 빈도 확인후 전처리
- 최소이하 제거 or 빈도가 지나치게 높은 counter에 대해 임의 제거
- 잘못 분리된 단어 확인하여 사용자단어에 추가
6. TF-IDF 해보기
7. 전처리 반복

- 완벽한 전처리 결론이 목요일까지 나오는 것이 목표

- 참고한 전처리
https://haystar.tistory.com/11

- hanspell 관련확인 _clone도 안됨. 해결 코드가 있긴한데 일단 후순위로 미루고 작업
https://velog.io/@woalsgodqhr/%EB%84%A4%EC%9D%B4%EB%B2%84-%EB%A7%9E%EC%B6%A4%EB%B2%95-%EA%B2%80%EC%82%AC%EA%B8%B0-api-%EC%9C%A0%ED%9A%A8%ED%95%9C-%ED%82%A4%EA%B0%80-%EC%95%84%EB%8B%99%EB%8B%88%EB%8B%A4-%EC%98%A4%EB%A5%98-%ED%95%B4%EA%B2%B0%ED%95%98%EA%B8%B0

In [ ]:
# 공지 csv의 결측치 파악
# df2[df2['content'] == '\n\n']

In [ ]:
import pandas as pd
import os

In [ ]:
#경로확인
print(os.getcwd())

/content


In [ ]:
new_dir = '/content'

# 작업 디렉토리 변경
os.chdir(new_dir)

### 마운트 및 파일 로드부터 시작하기 (경로 설정)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#kospacing 받기
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-0zv5napl
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-0zv5napl
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit 103ff614a2edf6df87d289ccaf99822b9afa9a42
  Preparing metadata (setup.py) ... done


In [ ]:
#작동확인
sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
new_sent = sent.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
print(new_sent)

In [ ]:
from pykospacing import Spacing
spacing = Spacing()
kospacing_sent = spacing(new_sent)
#결과일치시 맞음
print(sent)
print(kospacing_sent)

In [ ]:
#결과일치시 맞음
print(sent)
print(kospacing_sent)

In [ ]:
gongji = pd.read_csv('./drive/MyDrive/csv모음/merge_gongji.csv')
haksa = pd.read_csv('./drive/MyDrive/csv모음/merged_haksa.csv')
hiring = pd.read_csv('./drive/MyDrive/csv모음/merged_hire.csv')
janghak = pd.read_csv('./drive/MyDrive/csv모음/merged_janghak.csv')
#공지가 너무 커 세개로 나눔
gongji_1 = pd.read_csv('./drive/MyDrive/csv모음/gongji_1.csv')
gongji_2 = pd.read_csv('./drive/MyDrive/csv모음/gongji_2.csv', encoding = "cp949")
gongji_3 = pd.read_csv('./drive/MyDrive/csv모음/gongji_3.csv')
gongji.head(5)

# re 이용해 특정 문자열 제거

In [ ]:
# 원본 데이터 보존 위해 copy 데이터 생성
gongji_copy = gongji.copy()
haksa_copy = haksa.copy()
hiring_copy = hiring.copy()
janghak_copy = janghak.copy()


gongji1_copy = gongji_1.copy()
gongji2_copy = gongji_2.copy()
gongji3_copy = gongji_3.copy()

In [ ]:
haksa_copy.loc[5,'content']

### 전처리과정
- 전번, 주소, 메일 등 정보 부터 지우는 중
- 가독성 때문에 띄어쓰기 및 특수기호는 문자열 지운 이후에 마지막에 지울 듯 함
- 현재 test만 지웠고 데이터 프레임에 적용하지 않았음

In [ ]:
import re

In [ ]:
gongji_copy.loc[0,'content']

In [ ]:
# 전화번호를 포함하는 패턴
tel_number = r'TEL\.\d{2}-?\d{3,4}-\d{4}'
#전번 양식 (번호만 있는 경우대비)
phone_number = r'\d{2}-\d{4}-\d{4}'
#팩스 양식
fax_number = r'FAX\.\d{2}-\d{3}-\d{4}'
#주소 양식
address = r'서울시 동대문구 이문로 107'
#이메일 양식
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
#링크 양식
link_http = r'http://\S+'
link_www = r'\bwww\.\S*'
link_hufs = r'hufs\.ac\.kr'

#문장부호 지우기
punctuation_pattern = r'[,.:!?\\|※‘’”“→-]'

In [ ]:
test = gongji_copy.loc[0,'content']
print(test)

In [ ]:
# 각 패턴에 대해 검열 수행
test = re.sub(tel_number, '', test)
test = re.sub(phone_number, '', test)
test = re.sub(fax_number, '', test)
test = re.sub(address,'',test)
test = re.sub(email_pattern,'',test)
test = re.sub(link_http,'',test)
test = re.sub(link_www,'',test)
test = re.sub(link_hufs,'',test)
test = re.sub(punctuation_pattern,'',test)
print(test)

In [ ]:
def preprocess_text(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    #text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』]', '', text)

    return text


# 'content' 열에 전처리 함수를 적용하여 새로운 열 생성
gongji2_copy['cleaned_content'] = gongji2_copy['content'].apply(preprocess_text)

In [ ]:
print(gongji2_copy.loc[300, 'cleaned_content'])

In [ ]:
spacing = Spacing()

# 띄어쓰기 함수 적용
gongji2_copy['cleaned_content'] = gongji2_copy['cleaned_content'].apply(lambda x: spacing(x))

# 결과 확인
print( gongji2_copy.loc[300, 'cleaned_content'])
print( gongji2_copy.loc[300, 'cleaned_content'])


In [ ]:
# 결과 확인
print( gongji2_copy.loc[121, 'cleaned_content'])
print( gongji2_copy.loc[121, 'cleaned_content'])


### gongji2-> gongji_2_spacing.csv
- 기본 전처리 후 띄어쓰기 적용한 것 저장해둔 상태
- 전처리 좀 더 하고 품사태깅(okt)
- 빈도확인 및 사용자 사전 만들기
- TF_IDF 만들어보기

In [ ]:
# 결과를 CSV 파일로 저장
gongji2_copy.to_csv('gongji_2_spacing.csv', index=False)

# 저장된 파일 경로 확인
print("File saved asgongji_2_spacing.csv")

In [ ]:
pwd

In [ ]:
test = pd.read_csv('./drive/MyDrive/csv모음/gongji_2_spacing.csv')
test.head(10)

In [ ]:
test.loc[5,'cleaned_content']

In [ ]:
#복사
pre_gongji2_copy = test.copy()
print(pre_gongji2_copy.loc[501,'cleaned_content'])
print("####################################")
print(pre_gongji2_copy.loc[501,'content'])

### 0428~
- 정제된 텍스트 바탕으로 Okt뽑기시작

In [ ]:
#1. Konlpy에서 Okt를 불러옵니다.
!pip install konlpy
#2. konlpy 실행을 위한 자바어쩌구 다운
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#3. 필요한 설정
import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from konlpy.tag import Okt

okt = Okt()
okt.morphs("한국외국어대학교에서 안내드립니다. 오늘은 학교수업이 없습니다.", stem = True)

In [ ]:
#복사
pre_gongji2_copy = test.copy()
print(pre_gongji2_copy.loc[501,'cleaned_content'])
print("####################################")
print(pre_gongji2_copy.loc[501,'content'])

In [ ]:
# 부동 소수점 값을 문자열로 변환하여 데이터프레임 열의 모든 값이 문자열로 구성되도록 함
pre_gongji2_copy['cleaned_content'] = pre_gongji2_copy['cleaned_content'].astype(str)

# 변환된 문자열 값을 사용하여 단어 리스트 생성
words = " ".join(pre_gongji2_copy['cleaned_content'].tolist())
words = okt.morphs(words, stem=True)
print(len(words))

In [ ]:
from collections import Counter
frequent = Counter(words).most_common()
frequent


In [ ]:
# 결과를 텍스트 파일로 저장
with open('word_frequency.txt', 'w', encoding='utf-8') as f:
    for word, count in frequent:
        f.write(f'{word}: {count}\n')
from google.colab import files
files.download('word_frequency.txt')


### 학사
- 공지 프로세스 이해를 위한 전처리 및 Tf-IDF까지 일단 가보기
- 전처리 리스트가 어느정도 완성될것이라 기대하며ㅠㅠ

In [ ]:
haksa_copy

In [ ]:
import re

### 학사 전처리 유의사항 적는 곳
- 이중전공 사전에 넣어줄 것: 이중전공을 잘 인식못함
- 이수도 넣어주기 (이/수로 인식함) _ 이수, 이수구분
- 미배정도 미/배정으로 끊김
- 학년도를 학년/도(조사)로 인식함

In [ ]:
from pykospacing import Spacing
spacing = Spacing()
kospacing_sent = spacing(haksa_copy.loc[0,'content'])
print(haksa_copy.loc[0,'content'])
print(kospacing_sent)

In [ ]:
def preprocess_text(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    #text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    #마지막 날짜 양식 변형
    #text = re.sub(r'\d{4}\.\s\d{2}\.\s\d{2}\.', '', text)
    #마지막 날짜
    #text = re.sub(r'\d{4}\.\s\d{2}\.', '', text) #과제 제출등에 유의
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』)(]', '', text)
    #학사한정: 교무처장 제거
    text = re.sub(r'교무처장', '', text)
    text = re.sub(r'\n','',text)

    return text


# 'content' 열에 전처리 함수를 적용하여 새로운 열 생성
haksa_copy['cleaned_content'] = haksa_copy['content'].apply(preprocess_text)
haksa_copy.tail()

In [ ]:
print(haksa_copy.loc[118, 'cleaned_content'])
print("________________________________________")
print(haksa_copy.loc[118, 'content'])

In [ ]:
# kospacing 안하고 okt 넣어보기
#1. Konlpy에서 Okt를 불러옵니다.
!pip install konlpy
#2. konlpy 실행을 위한 자바어쩌구 다운
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#3. 필요한 설정
import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from konlpy.tag import Okt

okt = Okt()
okt.morphs("한국외국어대학교에서 안내드립니다. 오늘은 학교수업이 없습니다.", stem = True)

In [ ]:
#빈도확인용
okt = Okt()
words = " ".join( haksa_copy['cleaned_content'].tolist())
words = okt.morphs(words,stem=True)

In [ ]:
#품사태그로 쪼개주기
okt = Okt()
tagged_words = []
for sentence in haksa_copy['cleaned_content']:
    tagged_sentence = okt.pos(sentence, stem=True)
    tagged_words.append(tagged_sentence)

tagged_words

In [ ]:
#품사 없이 단어만
okt = Okt()
words_list = []

for sentence in haksa_copy['cleaned_content']:
    words = okt.morphs(sentence, stem=True)
    words_list.append(words)


In [ ]:
#태그된 단어 일단 메모장에넣어주기
from google.colab import files

with open('haksa_tagged_words.txt', 'w', encoding='utf-8') as file:
    for sentence in tagged_words:
        line = ' '.join([f'{word}/{tag}' for word, tag in sentence])
        file.write(line + '\n')

files.download('haksa_tagged_words.txt')


In [ ]:
#품사없이 단어만 일단 메모장에넣어주기
from google.colab import files

# 파일을 쓰기 모드로 열고 데이터를 쓴 후 파일을 다운로드합니다.
with open('haksa_words_list.txt', 'w', encoding='utf-8') as file:
    for words in words_list:
        line = ' '.join(words)
        file.write(line + '\n')

# 다운로드할 파일을 지정합니다.
files.download('haksa_words_list.txt')

In [ ]:
words[4]

In [ ]:
from collections import Counter
frequent = Counter(words).most_common()
frequent

### tf-idf 참고링크
https://heytech.tistory.com/337


In [ ]:
#tf-idf 넣어줄 사이킷런다운
!pip install -U scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#판다스로열어줌
documents = pd.read_csv("haksa_words_list.txt")
documents

In [ ]:
#공지 읽어오기
with open("haksa_words_list.txt",'r',encoding = 'utf-8') as file:
  lines = file.readlines()
  data = [line.strip() for line in lines]
documents = data
documents[1]

In [ ]:
vectorizer = CountVectorizer()
# Document Term Matrix
dtm = vectorizer.fit_transform(documents)

# Term Freqeuncy
tf = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names_out())
# Document Frequency
df = tf.astype(bool).sum(axis = 0)
# 문서 개수
D = len(tf)

# Inverse Document Frequency
idf = np.log((D) / (df+1))
# TF-IDF
tfidf = tf * idf
tfidf = tfidf / np.linalg.norm(tfidf, axis = 1, keepdims = True)
tfidf

In [ ]:
tf

In [ ]:
df

In [ ]:
D

In [ ]:
tfidf

# 코드 정리지점
### 다시 공지 전처리 해보고 tf-idf 해보기

In [ ]:
#호출코드모음
import pandas as pd
import os
#마운트
from google.colab import drive
drive.mount('/content/drive')
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
#단어 빈도셀 때
from collections import Counter

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#kospacing 다운 및 호출
#!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
#from pykospacing import Spacing
#spacing = Spacing()
#kospacing_sent = spacing(new_sent)

In [ ]:
#konlpy 다운 및 호출
#1. Konlpy에서 Okt
!pip install konlpy
#2. konlpy 실행을 위한 자바어쩌구 다운
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#3. 필요한 설정
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from konlpy.tag import Okt

okt = Okt()
okt.morphs("한국외국어대학교에서 안내드립니다. 오늘은 학교수업이 없습니다.", stem = True)


['한국외국어대학교', '에서', '안', '내', '드리다', '.', '오늘', '은', '학교', '수업', '이', '없다', '.']

In [ ]:
okt.morphs("안내 합니다")
okt.morphs("안내 드립니다")
okt.morphs("안내드립니다")

['안', '내', '드립니다']

In [ ]:
#각종 문서 저장

gongji = pd.read_csv('./drive/MyDrive/csv모음/final_merged_gongji(0501).csv')
haksa = pd.read_csv('./drive/MyDrive/csv모음/merged_haksa.csv')
hiring = pd.read_csv('./drive/MyDrive/csv모음/merged_hire.csv')
janghak = pd.read_csv('./drive/MyDrive/csv모음/merged_janghak.csv')

#공지 분할 저장
gongji_1 = pd.read_csv('./drive/MyDrive/csv모음/final_gongji_1.csv')
gongji_2 = pd.read_csv('./drive/MyDrive/csv모음/final_gongji_2.csv')
gongji_3 = pd.read_csv('./drive/MyDrive/csv모음/final_gongji_3.csv')
gongji_4 = pd.read_csv('./drive/MyDrive/csv모음/final_gongji_4.csv')

In [ ]:
#공지 4분할

# DataFrame 분할
'''splitted_dfs = np.array_split(gongji, 4)

# 각각의 DataFrame을 따로 저장할 수도 있습니다
for i, splitted_df in enumerate(splitted_dfs):
    splitted_df.to_csv(f'final_gongji_{i+1}.csv', index=False)'''

"splitted_dfs = np.array_split(gongji, 4)\n\n# 각각의 DataFrame을 따로 저장할 수도 있습니다\nfor i, splitted_df in enumerate(splitted_dfs):\n    splitted_df.to_csv(f'final_gongji_{i+1}.csv', index=False)"

In [ ]:
#각 공지 1677개씩 있음
len(gongji_4)

1677

In [ ]:
#파일복사
# 원본 데이터 보존 위해 copy 데이터 생성
gongji_copy = gongji.copy()
haksa_copy = haksa.copy()
hiring_copy = hiring.copy()
janghak_copy = janghak.copy()


gongji1_copy = gongji_1.copy()
gongji2_copy = gongji_2.copy()
gongji3_copy = gongji_3.copy()
gongji4_copy = gongji_4.copy()

In [ ]:
#전처리 함수(통일)
def preprocess_text(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    #text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』]', '', text)

    return text

# 학사처리
1. 기본 양식 re로 전처리
2. 코로나 게시글 제거 (제거한 데이터프레임 따로저장?)
3. 사용자 사전 추가
4. 빈도수 확인 후 사용자 사전 추가
5. 한글자 지우기
6. 품사태깅해서 조사, 동사 등등 지우기
7. TF-IDF 연산
8. 다시 사용자 사전 추가


In [ ]:
haksa = pd.read_csv('./drive/MyDrive/csv모음/merged_haksa.csv')
haksa_copy = haksa.copy()
haksa_copy

,index,date,title,content
0,1,2020-01-06,[공통] 2019-2학기 이중전공 변경 배정 확정 공고,\n\n\n\n\n\n[공통]이중전공변경 배정 확정 공고\n \n2019학년도 제2...
1,2,2020-01-13,[공통] 2020학년도 1학기 수강신청 공고,\n2020학년도 1학기 수강신청 공고 2020학년도 제1학기 수강신청에 대하여 아...
2,3,2020-01-13,[공통] 2020학년도 전기 국제지역대학원 학석사연계과정 모집 안내,\n2020학년도 전기 국제지역대학원 학석사연계과정 모집 안내\n \n2020학년도...
3,4,2020-01-13,[공통] 2020학년도 전기 일반대학원 학석사연계과정 모집 안내,\n2020학년도 전기 일반대학원 학석사연계과정 모집 안내\n\n2020학년도 전기...
4,5,2020-01-15,[글로벌]졸업 예비사정 결과 조회,\n[글로벌] 졸업 예비사정 결과 조회글로벌캠퍼스 졸업(2020년 2월) 예비사정 ...
...,...,...,...,...
617,618,2024-03-13,[공통] 생리 유고 신청 안내,\n\n\n\n\n\n생리 유고 신청 안내\n\n\n\n\n\n 생리로 인한 유고...
618,619,2024-03-13,[공통] 졸업예정자(해당학기 졸업가능자)의 조기취업 유고결석 신청 안내,\n\n\n\n\n\n졸업예정자(해당학기 졸업가능자)의 조기취업유고결석 신청 안내\...
619,621,2024-03-15,[글로벌]2024-1학기 폐강(2차) 공고(글로벌캠퍼스),\n\n\n\n\n\n2024-1학기 폐강(2차) 공고(글로벌캠퍼스)\n\n\n\n...
620,620,2024-03-15,[서울] 2024-1학기 폐강(2차) 공고(서울캠퍼스),\n\n\n\n\n\n2024-1학기 폐강(2차) 공고(서울캠퍼스)\n\n\n\n\...


In [ ]:
def preprocess_haksa(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    #text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    #마지막 날짜 양식 변형
    #text = re.sub(r'\d{4}\.\s\d{2}\.\s\d{2}\.', '', text)
    #마지막 날짜
    #text = re.sub(r'\d{4}\.\s\d{2}\.', '', text) #과제 제출등에 유의
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』)(]', '', text)
    text= re.sub(r'\n','',text)
    return text

In [ ]:
# 'content' 열에 전처리 함수를 적용하여 새로운 열 생성
haksa_copy['cleaned_content'] = haksa_copy['content'].apply(preprocess_haksa)
haksa_copy.loc[6,'cleaned_content']

'공    고 2020학년도 제1학기 휴·복학 신청 일정 및 방법 등을 공고합니다1 일정  2020 2 3월 ~ 2 7금2 방법  [종합정보시스템] 로그인  [휴/복학 신청]3 유의사항 가 일반 휴학 - 휴학은 1년 단위로 신청 단 1개 학기 이후 복학 가능 - 휴학기간 만료 후 다시 휴학신청 가능하며 재학 중 3년을 초과할 수 없음 단 2012년 2학기 이후 3학년으로 편입학한 학생은 2년을 초과할 수 없음  휴학기간 만료 후 복학하지 않을 경우 휴학기간초과제적됨 - 신·편입생 재입학생의 첫 학기 휴학 불가 단 질병으로 인해 4주 이상 치료를 요하는 경우에 한해서 일반휴학을 허가할 수 있음 별도문의 - 수업일수 1/4선 이후 아래의 경우에 한해서만 휴학을 허가하고 등록금 차액은 환불함  질병 또는 입원으로 인하여 4주 이상의 치료를 요하는 경우  3/4선까지만 허용 종합병원에서 발급받은 4주 이상 입원진단서 또는 4주 이상 수업참여가 어렵다는 의사소견진단서 제출  학업을 계속할 수 없는 명백한 사유가 있는 경우  중간고사 전 주 금요일까지만 허용 공식적인 서류로 사유를 증명할 수 있어야 함  증빙서류 미비 또는 요건 미충족 시 허가하지 않음 - 일반 휴학 중 입대할 경우 군입대 휴학으로 재신청하여야 함미처리 시 일반 휴학기간 종료 후 제적처리 됨 - 등록 휴학  휴학신청기간에 휴학 신청 후 등록기간에 등록금 납부 시 자동 처리됨 나 군입대 휴학 - 입대일 이전 학교 종합정보시스템에서 신청 후 입영통지서 사본 제출 이메일  군입대 휴학자 중 장기복무자는 입영통지서 제출 시 또는 장기확정 시 필히 구두로 신고하여야 함 - 휴학기간  입영일부터 전역일까지 귀향조치 시 관련 증빙서류 제출 후 조치 요망 - 전역일 이후 계속 휴학을 원하는 경우 복학예정일이 남아있더라도 일반 휴학으로 전환하여야 함 전역일이 기재되어있는 증빙서류 제출 - 1/4선 후~3/4선 이전 입대 예정자는 일반 휴학 신청 후 군입대 휴학으로 재신청 하여야 함 - 3/4선 후 입대 예정자가 금학기 

In [ ]:
#어떻게조개지는지확인
okt = Okt()
words = haksa_copy.loc[17,'cleaned_content']
words = okt.morphs(words,stem=True)
print(words, end = "")

['졸업', '대상자', '안내문', '1', '졸업식', '행사', '및', '학위증', '배부', '-', '졸업식', '행사', '는', '신종', '코로나바이러스', '로', '인하다', '취소', '확정', '되어다', '-', '학위증', '은', '졸업', '일자', '2020221', '당일', '오전', '10시', '부터', '학과', '사무실', '에서', '배부', '하다', '-', '졸업', '증명서', '는', '2020221', '오전', '부터', '발급', '가능하다', '문', '의전', '화', '0221732517', '인터넷', '종합정보시스템', '증명', '발급', '또는', '원스톱', '서비스', '센터', '발급', '-', '총학생회', '주관', '가운', '대여', '및', '기념', '사진', '촬영', '은', '총학생회', '에서', '별도', '공지', '예정', '이다', '문의', '전화', '02217327702', '졸업', '사정', '결과', '조회', '종합정보시스템', '로그인', '후', '첫', '화면', '의', '졸업', '사정', '정보', '-', '졸업', '확정', '되다', '학생', '[', '2020/02', '/', '21', ']', '졸업', '사정', '대상자', '/', '졸업', '확정자', '로', '나타남', '-', '졸업', '심사', '탈락', '학생', '[', '2020/02', '/', '21', ']', '졸업', '사정', '대상자', '/', '졸업', '사정', '탈락자', '로', '표기', '되다', '3', '참고', '사항', '-', '졸업', '사정', '탈락', '인', '학생', '의', '미비', '되다', '졸업요건', '은', '종합정보시스템', '->', '성적', '/', '학점', '/', '졸업', '관리', '->', '졸업', '예비', '사정', '결과', '조회', '에서', '확인', '-', '졸업

In [ ]:
#빈도확인용
okt = Okt()
words = " ".join( haksa_copy['cleaned_content'].tolist())
words = okt.morphs(words,stem=True)

frequent = Counter(words).most_common()
frequent

[('하다', 5271),
 ('전공', 3868),
 ('1', 3667),
 ('신청', 3648),
 ('2', 3400),
 ('에', 3313),
 ('-', 3214),
 ('을', 3171),
 ('학과', 2935),
 ('은', 2683),
 ('이', 2564),
 ('학기', 2513),
 ('수', 2473),
 ('의', 2453),
 ('/', 2303),
 ('및', 2269),
 ('대학', 2085),
 ('되다', 2056),
 ('3', 1943),
 ('학생', 1893),
 ('학점', 1789),
 ('수강신청', 1642),
 ('4', 1530),
 ('~', 1519),
 ('이중전공', 1515),
 ('경우', 1478),
 ('함', 1457),
 ('학년', 1453),
 ('으로', 1453),
 ('수강', 1425),
 ('변경', 1404),
 ('한', 1396),
 ('를', 1346),
 ('기간', 1322),
 ('졸업', 1294),
 ('에서', 1259),
 ('자', 1224),
 ('이수', 1196),
 ('6', 1150),
 ('후', 1149),
 ('과', 1132),
 ('가능', 1112),
 ('제출', 1098),
 ('불가', 1083),
 ('는', 1076),
 ('확인', 1064),
 ('도', 1030),
 ('금', 1013),
 ('시', 1002),
 ('통번역', 996),
 ('과목', 992),
 ('가', 988),
 ('중', 983),
 ('납부', 983),
 ('부', 978),
 ('5', 965),
 ('수업', 963),
 ('7', 904),
 ('로', 900),
 ('없다', 891),
 ('성적', 874),
 ('해당', 846),
 ('교과목', 825),
 ('배정', 818),
 ('제', 807),
 ('학부', 807),
 ('있다', 801),
 ('학교', 783),
 ('나', 775),
 ('까지', 767)

######################공통 구간#######################################

이 아래로 공지 병합 코딩진행

In [ ]:
gongji2_copy['title']

In [ ]:
#코로나 개수확인
corona_count = len(gongji2_copy[gongji2_copy['title'].str.contains('코로나19')])
# 결과 출력
print("코로나19가 포함된 데이터 개수:", corona_count)
print("삭제 전 데이터 개수:", len(gongji2_copy))

In [ ]:
#공지2 코로나 행 삭제
drop_covid_2 = gongji2_copy[gongji2_copy['title'].str.contains('코로나19')]
gongji2_copy.drop(drop_covid_2.index, inplace=True)
print("삭제 후 데이터 개수:", len(gongji2_copy))

In [ ]:
#공지1 코로나 행 삭제
#코로나 개수확인
corona_count = len(gongji1_copy[gongji1_copy['title'].str.contains('코로나19')])
# 결과 출력
print("코로나19가 포함된 데이터 개수:", corona_count)
print("삭제 전 데이터 개수:", len(gongji1_copy))

drop_covid_1 = gongji1_copy[gongji1_copy['title'].str.contains('코로나19')]
gongji1_copy.drop(drop_covid_1.index, inplace=True)
print("삭제 후 데이터 개수:", len(gongji1_copy))

In [ ]:
#'content' 열에 전처리 함수를 적용하여 새로운 열 생성
gongji2_copy['cleaned_content'] = gongji2_copy['content'].apply(preprocess_text)
gongji2_copy.tail()

In [ ]:
#빈도확인용
okt = Okt()
words = " ".join( gongji2_copy['cleaned_content'].tolist())
words = okt.morphs(words,stem=True)
words

In [ ]:
frequent = Counter(words).most_common()
frequent

In [ ]:
pwd

In [ ]:
#코로나 개수확인
corona_count = len(haksa_copy[haksa_copy['title'].str.contains('코로나19')])
# 결과 출력
print("코로나19가 포함된 데이터 개수:", corona_count)
print("삭제 전 데이터 개수:", len(haksa_copy))

코로나19가 포함된 데이터 개수: 6
삭제 전 데이터 개수: 622


# 사용자 사전 고치기
- 아래의 절차 따르고 맨 아래 data write 쪽 정정해주기
- - 사용자 사전 만들기 (colab)
https://sirzzang.github.io/ai/AI-konlpy-userdic/

In [ ]:
#사용자 사전 찾아가기
os.chdir('/usr/local/lib/python3.10/dist-packages/konlpy/java')
os.getcwd()

'/usr/local/lib/python3.10/dist-packages/konlpy/java'

In [ ]:
#os.makedirs('./temp_fix')
os.chdir('./temp_fix')
os.getcwd()

'/usr/local/lib/python3.10/dist-packages/konlpy/java/temp_fix'

In [ ]:
!jar xvf ../open-korean-text-2.1.0.jar

  created: META-INF/
 inflated: META-INF/MANIFEST.MF
  created: META-INF/maven/
  created: META-INF/maven/org.openkoreantext/
  created: META-INF/maven/org.openkoreantext/open-korean-text/
 inflated: META-INF/maven/org.openkoreantext/open-korean-text/pom.properties
 inflated: META-INF/maven/org.openkoreantext/open-korean-text/pom.xml
  created: org/
  created: org/openkoreantext/
  created: org/openkoreantext/processor/
  created: org/openkoreantext/processor/tokenizer/
 inflated: org/openkoreantext/processor/tokenizer/Sentence.class
 inflated: org/openkoreantext/processor/tokenizer/KoreanSentenceSplitter.class
 inflated: org/openkoreantext/processor/tokenizer/KoreanDetokenizer.class
 inflated: org/openkoreantext/processor/tokenizer/KoreanChunker$ChunkMatch$.class
 inflated: org/openkoreantext/processor/tokenizer/KoreanTokenizer$.class
 inflated: org/openkoreantext/processor/tokenizer/KoreanSentenceSplitter$.class
 inflated: org/openkoreantext/processor/tokenizer/TokenizerProfile.class

In [ ]:
pwd

'/usr/local/lib/python3.10/dist-packages/konlpy/java/temp_fix'

In [ ]:
# 사용자 사전 열기
with open(f"/usr/local/lib/python3.10/dist-packages/konlpy/java/temp_fix/org/openkoreantext/processor/util/noun/names.txt") as f:
    data = f.read()
data

'가몽\n가온\n갓세븐\n강새이\n게임닉가\n관우\n귀여미\n규\n김유이\n김준면\n까까런\n노컷\n누너예\n니노\n다마고치\n다이무스\n대학생\n데이브\n도요토미\n동운\n동이\n두주니\n디시인사이드\n디오\n라몹\n라스\n라옵\n멍구\n메이든\n명덕\n명량\n문민정부\n미네\n방엘리\n병헌\n붓다\n비정상회담\n빼빼로\n삼풍\n샤인온미\n성식\n성열\n세라문\n세라복\n세종대왕\n손권\n손책\n쇼미더머니\n쇼챔\n순규\n스라소니\n신동아\n신쓰패밀리\n신아라\n아베\n안상홍\n안홍준\n여누\n여랑\n여포\n연합\n오꾸닭\n요섭\n웃찾사\n원식\n유병언\n유비\n유이\n윤기형\n이나단\n이명박\n이완용\n임창용\n자괴\n자니윤\n자대련\n자유\n재중이\n전교조\n정윤회\n제갈량\n조자룡\n조조\n준면\n지오디\n지존파\n진영오\n차작가\n차트\n창섭\n챠트\n첸\n코르사주\n하무열\n하용파쿠\n혁재\n현이\n현태\n혜미\n한국외국어대학교\n한국외대\n안내\\종합정보시스템\\공고\n이수\n학기\n대하여\n아래\n수강편람\n이중전공\n수강신청함\n수강신청\n예비수강신청\n교무처장\n수강료\n등록기간\n수강신청자\n한하다\n미수강신청자\n납부불가\n핫링크\n고지서출력\n가상계좌\n붙임파일\n모바일앱\n내역\n인터넷뱅킹\n텔레뱅킹\n개시일\n신청\n마감\n마감일\n수강취소\n수강변경\\종합정보화면\n계절학기\n부분환불\n계절학기\\학년도\n접수일\n학석사\n학사\n석사\n연계\n과정생\n접수처\n학사종합지원센터\n양캠퍼스\n통번역\n이수\n미배정\n부전공\\여석\n이수\n지원\n학사종합지원센터\n한국예술종합학교\n특성화\n수강편람\n실용외국어\n공동교양\n여석\\졸업\n사정\n결과\n확정자\n불가자\n개강일\n우리대학교\n우리대학\n안내문\n학위\n학위증\n코로나바이러스\nHUFS\n양캠퍼스\n서울캠퍼스\n글로벌캠퍼스\n서울글로벌캠퍼스\n서울·글로벌캠퍼스\n우리은행\n단과대학\n졸업시험\n논문\n제적\n졸업요건\n지역학사\n지역전문가과정\n수료졸업대기\nELLT학과

In [ ]:
data += "서울·글로벌캠퍼스\n우리은행\n단과대학\n졸업시험\n논문\n제적\n졸업요건\n지역학사\n지역전문가과정\n수료졸업대기\nELLT학과\n영어학과\n아랍어과\n미디어커뮤니케이션학부\n영어전공\n영미문학문화학과\n태국어과\n경제학부\nEICC학과\n영어통번역학과\n베트남어과\n국제통상학과\이란어과\n경영학부\n프랑스어학부\n인도어과\n영어교육과\n독일어과\n터키아제르바이잔어과\n프랑스어교육과\n노어과\n몽골어과\n독일어교육과\n스페인어과\n중국언어문화학부\n한국어교육과\n이탈리아어과\n중국외교통상학부\n국제학부\n포르투갈어과\n일본언어문화학부\n사회과학전공\n네덜란드어과\n융합일본지역학부\nLD전공\n스칸디나비아어과\n정치외교학과\nLT전공\n말레이인도네시아어과\n행정학과\n서양어대학\n아시아언어문화대학\n중국어대학\n일본어대학\n사회과학\n상경\n경영\n국제학부\n사회과학전공\n사범대학\n교직\n실용외국어\n군입대\n입영통지서\n미비\n미충족\n등록휴학\n휴학\n전역일\n증빙서류\n"
# 파일 새롭게 저장
with open("/usr/local/lib/python3.10/dist-packages/konlpy/java/temp_fix/org/openkoreantext/processor/util/noun/names.txt", 'w') as f:
    f.write(data)
data

'가몽\n가온\n갓세븐\n강새이\n게임닉가\n관우\n귀여미\n규\n김유이\n김준면\n까까런\n노컷\n누너예\n니노\n다마고치\n다이무스\n대학생\n데이브\n도요토미\n동운\n동이\n두주니\n디시인사이드\n디오\n라몹\n라스\n라옵\n멍구\n메이든\n명덕\n명량\n문민정부\n미네\n방엘리\n병헌\n붓다\n비정상회담\n빼빼로\n삼풍\n샤인온미\n성식\n성열\n세라문\n세라복\n세종대왕\n손권\n손책\n쇼미더머니\n쇼챔\n순규\n스라소니\n신동아\n신쓰패밀리\n신아라\n아베\n안상홍\n안홍준\n여누\n여랑\n여포\n연합\n오꾸닭\n요섭\n웃찾사\n원식\n유병언\n유비\n유이\n윤기형\n이나단\n이명박\n이완용\n임창용\n자괴\n자니윤\n자대련\n자유\n재중이\n전교조\n정윤회\n제갈량\n조자룡\n조조\n준면\n지오디\n지존파\n진영오\n차작가\n차트\n창섭\n챠트\n첸\n코르사주\n하무열\n하용파쿠\n혁재\n현이\n현태\n혜미\n한국외국어대학교\n한국외대\n안내\\종합정보시스템\\공고\n이수\n학기\n대하여\n아래\n수강편람\n이중전공\n수강신청함\n수강신청\n예비수강신청\n교무처장\n수강료\n등록기간\n수강신청자\n한하다\n미수강신청자\n납부불가\n핫링크\n고지서출력\n가상계좌\n붙임파일\n모바일앱\n내역\n인터넷뱅킹\n텔레뱅킹\n개시일\n신청\n마감\n마감일\n수강취소\n수강변경\\종합정보화면\n계절학기\n부분환불\n계절학기\\학년도\n접수일\n학석사\n학사\n석사\n연계\n과정생\n접수처\n학사종합지원센터\n양캠퍼스\n통번역\n이수\n미배정\n부전공\\여석\n이수\n지원\n학사종합지원센터\n한국예술종합학교\n특성화\n수강편람\n실용외국어\n공동교양\n여석\\졸업\n사정\n결과\n확정자\n불가자\n개강일\n우리대학교\n우리대학\n안내문\n학위\n학위증\n코로나바이러스\nHUFS\n양캠퍼스\n서울캠퍼스\n글로벌캠퍼스\n서울글로벌캠퍼스\n서울·글로벌캠퍼스\n우리은행\n단과대학\n졸업시험\n논문\n제적\n졸업요건\n지역학사\n지역전문가과정\n수료졸업대기\nELLT학과

In [ ]:
data

'가몽\n가온\n갓세븐\n강새이\n게임닉가\n관우\n귀여미\n규\n김유이\n김준면\n까까런\n노컷\n누너예\n니노\n다마고치\n다이무스\n대학생\n데이브\n도요토미\n동운\n동이\n두주니\n디시인사이드\n디오\n라몹\n라스\n라옵\n멍구\n메이든\n명덕\n명량\n문민정부\n미네\n방엘리\n병헌\n붓다\n비정상회담\n빼빼로\n삼풍\n샤인온미\n성식\n성열\n세라문\n세라복\n세종대왕\n손권\n손책\n쇼미더머니\n쇼챔\n순규\n스라소니\n신동아\n신쓰패밀리\n신아라\n아베\n안상홍\n안홍준\n여누\n여랑\n여포\n연합\n오꾸닭\n요섭\n웃찾사\n원식\n유병언\n유비\n유이\n윤기형\n이나단\n이명박\n이완용\n임창용\n자괴\n자니윤\n자대련\n자유\n재중이\n전교조\n정윤회\n제갈량\n조자룡\n조조\n준면\n지오디\n지존파\n진영오\n차작가\n차트\n창섭\n챠트\n첸\n코르사주\n하무열\n하용파쿠\n혁재\n현이\n현태\n혜미\n한국외국어대학교\n한국외대\n안내\\종합정보시스템\\공고\n이수\n학기\n대하여\n아래\n수강편람\n이중전공\n수강신청함\n수강신청\n예비수강신청\n교무처장\n수강료\n등록기간\n수강신청자\n한하다\n미수강신청자\n납부불가\n핫링크\n고지서출력\n가상계좌\n붙임파일\n모바일앱\n내역\n인터넷뱅킹\n텔레뱅킹\n개시일\n신청\n마감\n마감일\n수강취소\n수강변경\\종합정보화면\n계절학기\n부분환불\n계절학기\\학년도\n접수일\n학석사\n학사\n석사\n연계\n과정생\n접수처\n학사종합지원센터\n양캠퍼스\n통번역\n이수\n미배정\n부전공\\여석\n이수\n지원\n학사종합지원센터\n한국예술종합학교\n특성화\n수강편람\n실용외국어\n공동교양\n여석\\졸업\n사정\n결과\n확정자\n불가자\n개강일\n우리대학교\n우리대학\n안내문\n학위\n학위증\n코로나바이러스\nHUFS\n양캠퍼스\n서울캠퍼스\n글로벌캠퍼스\n서울글로벌캠퍼스\n서울·글로벌캠퍼스\n우리은행\n단과대학\n졸업시험\n논문\n제적\n졸업요건\n지역학사\n지역전문가과정\n수료졸업대기\nELLT학과

In [ ]:
!jar cvf ../open-korean-text-2.1.0.jar *

added manifest
ignoring entry META-INF/
ignoring entry META-INF/MANIFEST.MF
adding: META-INF/maven/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/pom.properties(in = 119) (out= 110)(deflated 7%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/pom.xml(in = 9127) (out= 2208)(deflated 75%)
adding: org/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/tokenizer/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/tokenizer/Sentence.class(in = 5985) (out= 3125)(deflated 47%)
adding: org/openkoreantext/processor/tokenizer/KoreanSentenceSplitter.class(in = 1092) (out= 712)(deflated 34%)
adding: org/openkoreantext/processor/tokenizer/Kore

In [ ]:
import os
from konlpy.tag import Okt
new_dir = '/content'

# 작업 디렉토리 변경
os.chdir(new_dir)
okt = Okt()
print(okt.nouns('송이레만세')) # ['송이레만세']
print(okt.morphs('김재경자경')) # ['김재경자경']
print(okt.morphs("한국외국어대학교"))

['송이', '레만', '세']
['김재경', '자', '경']
['한국외국어대학교']


In [ ]:
pwd

'/content'

# ---------------------------------------------------------- 사전 수정하는 법 --------------------------------------------------

# ------------------------- 수연님 공지 병합하기 ----------------------
- 하단 계속

In [ ]:
import pandas as pd

In [ ]:
gongji1_copy

In [ ]:
len(gongji1_copy)

1677

In [ ]:
gongji_2.loc[150]

index                                                   1977
date                                              2020-10-16
title                          코로나19 발병현황 및 행동지침(2020.10.16)
content    \n코로나19 발병현황 및 행동지침(2020.10.16)1. 일일 확진자 현황 (2...
Name: 150, dtype: object

In [ ]:
gongji_3 =  pd.read_csv('./drive/MyDrive/csv모음/gongji_fix/noti_gongji3.csv')
len(gongji_3)

352

In [ ]:
gongji_1 = pd.read_csv('./drive/MyDrive/csv모음/gongji_fix/merge_file_final(병합전).csv')
len(gongji_1)

4590

In [ ]:
# 세 개의 DataFrame을 하나로 병합
merged_df = pd.concat([gongji_1, gongji_2, gongji_3], ignore_index=True)

# 병합된 DataFrame을 새로운 인코딩으로 저장
merged_df.to_csv('merged_gongji_test.csv', encoding='utf-8-sig', index=False)

len(merged_df)

6708

In [ ]:
test = pd.read_csv("/content/merged_gongji_test.csv")

In [ ]:
test.loc[5000]

index                                                   1717
date                                              2020-06-16
title                                  외국인유학생회(GSA) 신입 스탭 모집
content    \n\n\n외국인유학생회(GSA) 신입 스탭 모집\n\n\n국제교류처 산하 외국인유...
Name: 5000, dtype: object

# 번역기 사용하여 외국어처리하기


In [ ]:
!pip install googletrans==3.1.0a0

# 한국어, 영어 문자만 남기기
from googletrans import Translator

def clean_text(text):
    translator = Translator()
    detected = translator.detect(text)
    lang = detected.lang
    if lang in ['ko', 'en']:
        return text
    else:
        return ''

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 8.6 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=b7caea3f3653e03523513809b9f8935483088e1dcc9b8ea50809fc23d84f87d1
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling c

In [ ]:
gongji1_copy['content'] = gongji1_copy['content'].apply(clean_text)

WriteError: EOF occurred in violation of protocol (_ssl.c:2426)